In [1]:
script_folder = @__DIR__ # folder where this script is located
data_path = joinpath(script_folder, "data") 

using Revise,DelimitedFiles,Random,StaticArrays
using ParallelTemperingMonteCarlo,StructArrays
using BenchmarkTools


Ensemble, potential, params etc

In [12]:
n_atoms = 38

ti = 300.
tf = 1300.
n_traj = 8

temp = TempGrid{n_traj}(ti,tf) 

mc_cycles = 100
mc_sample = 1

displ_atom = 0.1 # Angstrom
n_adjust = 100

max_displ_atom = [0.1*sqrt(displ_atom*temp.t_grid[i]) for i in 1:n_traj]

mc_params = MCParams(mc_cycles, n_traj, n_atoms, mc_sample = mc_sample, n_adjust = n_adjust)

evtohartree = 0.0367493
nmtobohr = 18.8973

#parameters taken from L Vocadlo etal J Chem Phys V120N6 2004
n = 8.482
m = 4.692
ϵ = evtohartree*0.0370
a = 0.25*nmtobohr
C = 27.561

pot = EmbeddedAtomPotential(n,m,ϵ,C,a)
#ensemble = NVT(n_atoms)
move_strat = MoveStrategy(ensemble)



pos_cu38 = [[2.3603476948363165, 2.3603476948363165, 0.0],
[6.994369407022418, 2.33998871029911, 0.0],
[2.33998871029911, 6.994369407022418, 0.0],
[-2.3603476948363165, 2.3603476948363165, 0.0],
[-6.994369407022418, 2.33998871029911, 0.0],
[-2.33998871029911, 6.994369407022418, 0.0],
[-2.3603476948363165, -2.3603476948363165, 0.0],
[-6.994369407022418, -2.33998871029911, 0.0],
[-2.33998871029911, -6.994369407022418, 0.0],
[2.3603476948363165, -2.3603476948363165, 0.0],
[6.994369407022418, -2.33998871029911, 0.0],
[2.33998871029911, -6.994369407022418, 0.0],
[0.0, 0.0, 3.3380357219419614],
[4.84532317769689, 0.0, 3.4261608756649893],
[-4.84532317769689, 0.0, 3.4261608756649893],
[0.0, 4.84532317769689, 3.4261608756649893],
[0.0, -4.84532317769689, 3.4261608756649893],
[4.667179058660764, 4.667179058660764, 3.2911441531516483],
[-4.667179058660764, 4.667179058660764, 3.2911441531516483],
[-4.667179058660764, -4.667179058660764, 3.2911441531516483],
[4.667179058660764, -4.667179058660764, 3.2911441531516483],
[0.0, 0.0, -3.3380357219419614],
[4.84532317769689, 0.0, -3.4261608756649893],
[-4.84532317769689, 0.0, -3.4261608756649893],
[0.0, 4.84532317769689, -3.4261608756649893],
[0.0, -4.84532317769689, -3.4261608756649893],
[4.667179058660764, 4.667179058660764, -3.2911441531516483],
[-4.667179058660764, 4.667179058660764, -3.2911441531516483],
[-4.667179058660764, -4.667179058660764, -3.2911441531516483],
[4.667179058660764, -4.667179058660764, -3.2911441531516483], 
[2.327190348361654, 2.327190348361654, 6.600387922922003],
[-2.327190348361654, 2.327190348361654, 6.600387922922003],
[-2.327190348361654, -2.327190348361654, 6.600387922922003],
[2.327190348361654, -2.327190348361654, 6.600387922922003],
[2.327190348361654, 2.327190348361654, -6.600387922922003],
[-2.327190348361654, 2.327190348361654, -6.600387922922003],
[-2.327190348361654, -2.327190348361654, -6.600387922922003],
[2.327190348361654, -2.327190348361654, -6.600387922922003]]
shuffle!(pos_cu38)
n_bin = 100
AtoBohr = 1.8897259886

bc_cu38 = SphericalBC(radius=14*AtoBohr) 
start_config = Config(pos_cu38, bc_cu38)


Config{38, SphericalBC{Float64}, Float64}(SVector{3, Float64}[[-4.84532317769689, 0.0, -3.4261608756649893], [-4.667179058660764, -4.667179058660764, 3.2911441531516483], [-2.327190348361654, 2.327190348361654, 6.600387922922003], [0.0, 0.0, -3.3380357219419614], [2.3603476948363165, 2.3603476948363165, 0.0], [0.0, -4.84532317769689, 3.4261608756649893], [-4.667179058660764, 4.667179058660764, -3.2911441531516483], [-2.327190348361654, -2.327190348361654, 6.600387922922003], [2.3603476948363165, -2.3603476948363165, 0.0], [0.0, 4.84532317769689, 3.4261608756649893]  …  [-6.994369407022418, -2.33998871029911, 0.0], [2.327190348361654, 2.327190348361654, -6.600387922922003], [6.994369407022418, -2.33998871029911, 0.0], [2.33998871029911, 6.994369407022418, 0.0], [4.667179058660764, -4.667179058660764, -3.2911441531516483], [6.994369407022418, 2.33998871029911, 0.0], [2.327190348361654, 2.327190348361654, 6.600387922922003], [-4.667179058660764, 4.667179058660764, 3.2911441531516483], [4.

RunnerPotential

We require a different view for our calculation of tpz and g_offset

In [13]:
X = [ 11              0.001   0.000  11.338
 10              0.001   0.000  11.338
 11              0.020   0.000  11.338
 10              0.020   0.000  11.338
 11              0.035   0.000  11.338
 10              0.035   0.000  11.338
 11              0.100   0.000  11.338
 10              0.100   0.000  11.338
 11              0.400   0.000  11.338
 10              0.400   0.000  11.338]
radsymmvec = []
V = [[0.0001,1,1,11.338],[0.0001,-1,2,11.338],[0.003,-1,1,11.338],[0.003,-1,2,11.338],[0.008,-1,1,11.338],[0.008,-1,2,11.338],[0.008,1,2,11.338],[0.015,1,1,11.338],[0.015,-1,2,11.338],[0.015,-1,4,11.338],[0.015,-1,16,11.338],[0.025,-1,1,11.338],[0.025,1,1,11.338],[0.025,1,2,11.338],[0.025,-1,4,11.338],[0.025,-1,16,11.338],[0.025,1,16,11.338],[0.045,1,1,11.338],[0.045,-1,2,11.338],[0.045,-1,4,11.338],[0.045,1,4,11.338],[0.045,1,16,11.338],[0.08,1,1,11.338],[0.08,-1,2,11.338],[0.08,-1,4,11.338],[0.08,1,4,11.338]]
T = [111,110,100]
angularsymmvec = []
#-------------------------------------------#
#-----------Including scaling data----------#
#-------------------------------------------#
file = open(joinpath(data_path,"scaling.data")) # full path "./data/scaling.data"
scalingvalues = readdlm(file)
close(file)
G_value_vec = []
for row in eachrow(scalingvalues[1:88,:])
    max_min = [row[4],row[3]]
    push!(G_value_vec,max_min)
end
for symmindex in eachindex(eachrow(X))
    row = X[symmindex,:]
    radsymm = RadialType2{Float64}(row[2],row[4],Int(row[1]),G_value_vec[symmindex])
    push!(radsymmvec,radsymm)
end
let n_index = 10
for element in V
    for types in T

        n_index += 1

        symmfunc = AngularType3{Float64}(element[1],element[2],element[3],11.338,types,G_value_vec[n_index])

        push!(angularsymmvec,symmfunc)
    end
end
end

totalsymmvec = vcat(radsymmvec,angularsymmvec)
num_nodes::Vector{Int32} = [88, 20, 20, 1]
activation_functions::Vector{Int32} = [1, 2, 2, 1]
file = open(joinpath(data_path, "weights.029.data"), "r+") # "./data/weights.029.data"
weights=readdlm(file)
close(file)
weights = vec(weights)
nnp = NeuralNetworkPotential(num_nodes,activation_functions,weights)
testpotential = RuNNerPotential(nnp,radsymmvec,angularsymmvec)
#statest,stratest,resultest,nsteptest,startcounttest=initialisation(mc_params,temp,start_config,testpotential,ensemble)

RuNNerPotential{10, 78}(NeuralNetworkPotential(4, 2221, Int32[88, 20, 20, 1], Int32[1, 2, 2, 1], [-1.0768943782, 0.3563458393, -1.3084861447, 0.1127640916, -0.5646591931, -0.4969830793, 1.423770514, -0.2005662393, -0.8957859374, 0.6076110858  …  0.1939524848, -0.0342903191, -0.0231042009, 0.0718481968, -0.0828320122, -0.0728501885, 0.0306628826, 0.0486053813, -0.0867018862, 0.0441651841]), RadialType2{Float64}[RadialType2{Float64}(0.001, 11.338, 11, -0.029892621990594427, 0.06634475788973929), RadialType2{Float64}(0.001, 11.338, 10, -0.0, 0.09828103647521905), RadialType2{Float64}(0.02, 11.338, 11, -0.013430592462258787, 0.13321466604629645), RadialType2{Float64}(0.02, 11.338, 10, -0.0, 0.18618835781674212), RadialType2{Float64}(0.035, 11.338, 11, -0.0067056953127401205, 0.2093521117141925), RadialType2{Float64}(0.035, 11.338, 10, -0.0, 0.28571045376567983), RadialType2{Float64}(0.1, 11.338, 11, -0.00026435144512647774, 0.9364998977117353), RadialType2{Float64}(0.1, 11.338, 10, -0.0, 1

In [14]:
X = [ 2              0.001   0.000  11.338
 #10              0.001   0.000  11.338
 2              0.020   0.000  11.338
 #10              0.020   0.000  11.338
 2              0.035   0.000  11.338
 #10              0.035   0.000  11.338
 2              0.100   0.000  11.338
 #10              0.100   0.000  11.338
 2              0.400   0.000  11.338
 #10              0.400   0.000  11.338
 ]

radsymmvec = []
#--------------------------------------------#
#--------Vector of angular symm values-------#
#--------------------------------------------#
V = [[0.0001,1,1,11.338],
[0.0001,-1,2,11.338],
[0.003,-1,1,11.338],
[0.003,-1,2,11.338],
[0.008,-1,1,11.338],
[0.008,-1,2,11.338],
[0.008,1,2,11.338],
[0.015,1,1,11.338],
[0.015,-1,2,11.338],
[0.015,-1,4,11.338],
[0.015,-1,16,11.338],
[0.025,-1,1,11.338],
[0.025,1,1,11.338],
[0.025,1,2,11.338],
[0.025,-1,4,11.338],
[0.025,-1,16,11.338],
[0.025,1,16,11.338],
[0.045,1,1,11.338],
[0.045,-1,2,11.338],
[0.045,-1,4,11.338],
[0.045,1,4,11.338],
[0.045,1,16,11.338],
[0.08,1,1,11.338],
[0.08,-1,2,11.338],
[0.08,-1,4,11.338],
[0.08,1,4,11.338]]
T = [111,110,100]
angularsymmvec = []
#-------------------------------------------#
#-----------Including scaling data----------#
#-------------------------------------------#
file = open(joinpath(data_path,"scaling.data")) # full path "./data/scaling.data"
scalingvalues = readdlm(file)
close(file)
G_value_vec = []
for row in eachrow(scalingvalues[1:88,:])
    max_min = [row[4],row[3]]
    push!(G_value_vec,max_min)
end
G_value_vec_b = []
for row in eachrow(scalingvalues[89:end,:])
    max_min = [row[4],row[3]]
    push!(G_value_vec_b,max_min)
end
for symmindex in eachindex(eachrow(X))
    row = X[symmindex,:]
    radsymm = RadialType2a{Float64}(row[2],row[4],Int(row[1]),[G_value_vec[(symmindex-1)*2 + 1],G_value_vec[(symmindex-1)*2 + 2]],[G_value_vec_b[(symmindex-1)*2 + 1],G_value_vec_b[(symmindex-1)*2 + 2]])
    push!(radsymmvec,radsymm)
end
let n_index = 10
let j_index = 0
for element in V
    #for types in T

        j_index += 1

        symmfunc = AngularType3a{Float64}(element[1],element[2],element[3],11.338,2,[G_value_vec[n_index + (j_index-1)*3 + 1],G_value_vec[n_index + (j_index-1)*3 + 2],G_value_vec[n_index + (j_index-1)*3 + 3] ],[G_value_vec_b[n_index + (j_index-1)*3 + 1],G_value_vec_b[n_index + (j_index-1)*3 + 2],G_value_vec_b[n_index + (j_index-1)*3 + 3] ])

        push!(angularsymmvec,symmfunc)
    #end
end
end
end
#---------------------------------------------------#
#------concatenating radial and angular values------#
#---------------------------------------------------#
totalsymmvec = vcat(radsymmvec,angularsymmvec)
#--------------------------------------------------#
#-----------Initialising the nnp weights-----------#
#--------------------------------------------------#
num_nodes::Vector{Int32} = [88, 20, 20, 1]
activation_functions::Vector{Int32} = [1, 2, 2, 1]


4-element Vector{Int64}:
 1
 2
 2
 1

In [15]:
file = open(joinpath(data_path, "weights.029.data"), "r+") # "./data/weights.029.data"
weights=readdlm(file)
close(file)
weights = vec(weights)
nnpcu = NeuralNetworkPotential(num_nodes,activation_functions,weights)

file2=open(joinpath(data_path, "weights.030.data"),"r+") #./data/weights.030.data
weights2=readdlm(file2)
close(file2)
weights2 = vec(weights2)
nnpzn= NeuralNetworkPotential(num_nodes,activation_functions,weights2)
ensemble = NNVT([32,6];natomswaps=2)

runnerpotential = RuNNerPotential2Atom(nnpcu,nnpzn,radsymmvec,angularsymmvec,32,6)

RuNNerPotential2Atom{5, 26, 32, 6}(NeuralNetworkPotential(4, 2221, Int32[88, 20, 20, 1], Int32[1, 2, 2, 1], [-1.0768943782, 0.3563458393, -1.3084861447, 0.1127640916, -0.5646591931, -0.4969830793, 1.423770514, -0.2005662393, -0.8957859374, 0.6076110858  …  0.1939524848, -0.0342903191, -0.0231042009, 0.0718481968, -0.0828320122, -0.0728501885, 0.0306628826, 0.0486053813, -0.0867018862, 0.0441651841]), NeuralNetworkPotential(4, 2221, Int32[88, 20, 20, 1], Int32[1, 2, 2, 1], [-0.2990596476, 0.50210302, -0.0769338744, -1.1006272074, 0.2955859359, 1.7429579447, -1.6013368538, 0.2350663304, 0.5352038521, 0.0051334333  …  -0.07023029, -0.0274223406, 0.0379881304, 0.1032826216, -0.0671415566, 0.0873441204, 0.0024981831, -0.1005392202, 0.011513595, -0.0270711056]), RadialType2a{Float64}[RadialType2a{Float64}(0.001, 11.338, 2, [-0.029892621990594427, -0.0, -0.0, -0.0], [0.06634475788973929, 0.09828103647521905, 0.07273034544948624, 0.07540689868304344]), RadialType2a{Float64}(0.02, 11.338, 2, [-

In [16]:
X = ptmc_run!(mc_params,temp,start_config,runnerpotential,ensemble;save=100)

params set
equilibration complete
MC loop done.
[36.50900319507937, 55.26453955529784, 8.045056600941164, 45.13026245849824, 95.90940526162203, 56.44629312065602, 59.491488949965934, 27.435834024064263]
done


(MCState{Float64, 38, SphericalBC{Float64}, NNPVariables2a{Float64, 38, 88}, NNVTVariables{Float64, 38, 32, 6}}[MCState{Float64, 38, SphericalBC{Float64}, NNPVariables2a{Float64, 38, 88}, NNVTVariables{Float64, 38, 32, 6}}(300.0, 1052.5832842103239, Config{38, SphericalBC{Float64}, Float64}(SVector{3, Float64}[[-5.132751092106602, 0.14097705879059189, -3.6623090087808414], [-7.342454481377733, 2.538079427745496, 0.06289571678818986], [-4.602936487625598, 4.782958891637526, 2.978697165821442], [0.09164333343583415, -0.24224671017661467, -3.5144566403567747], [2.2380657363296885, 2.1399166333581463, -0.03937949322628198], [-0.0008572611887896452, -5.224167837890527, -3.6586622418850636], [-4.6520297423547765, 4.6593040168074635, -3.6449954449455784], [-2.327922530208495, -2.5843079414581913, -6.619822977274905], [2.3044879736611423, -2.5166129781905684, 0.07074599638237783], [-0.0785007327022782, 5.160820640123689, 3.4539588077565955]  …  [4.823624958162985, 4.8445568798567304, -3.267759

In [17]:
isa(runnerpotential,AbstractMachineLearningPotential)

true

In [18]:
runnerpot = RuNNerPotential2Atom(nnpcu,nnpzn,radsymmvec,angularsymmvec,4,2)
v1 = SVector(2.36, 2.36, 0.0)
v2 = SVector(6.99, 2.33, 0.0)
v3 = SVector(2.33, 6.99, 0.0)
v4 = SVector(-2.36, 2.36, 0.0)
v5 = SVector(-6.99, 2.33, 0.0)
v6 = SVector(-2.33, 6.99, 0.0)
bc = SphericalBC(radius=2.0)
conf = Config{6}([v1,v2,v3,v4,v5,v6],bc)
d2mat = get_distance2_mat(conf)
vars = set_variables(conf,d2mat,runnerpot)
evars = set_ensemble_variables(conf,NNVT([4,2]))

NNVTVariables{Float64, 6, 4, 2}(1, [0.0, 0.0, 0.0], [1, 5])

In [19]:
isa(vars.g_matrix,MMatrix)
E,vars = initialise_energy(conf,d2mat,vars,evars,runnerpot)

(-0.2616528993354795, NNPVariables2a{Float64, 6, 88}([-0.07961990236909662, -0.046329508101558725, -0.06652413358790898, -0.08616269292006679, 0.012197910417455157, 0.0047854272256963905], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.09435530394181704 0.040032613593324516 … 0.0516772471800642 0.1181785536901605; 0.041829635204485544 0.0014182473523196017 … 0.02698557697421576 0.02698557697421576; … ; 0.012001828644531058 4.708405159025907e-7 … 0.007679720869687302 0.007679027976057983; 7.081025391239545e-7 0.0 … 0.0 0.0], [1.0 0.6419526956120072 … 0.07395536545379516 0.3737328615585688; 0.6419526956120072 1.0 … 0.0 0.016085593143803867; … ; 0.07395536545379516 0.0 … 1.0 0.37375116838713807; 0.3737328615585688 0.016085593143803867 … 0.37375116838713807 1.0], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]))

In [20]:
E ≈-0.261652899

true

In [105]:
v1 = SVector(2.36, 2.36, 0.0)
v2 = SVector(6.99, 2.33, 0.0)
v3 = SVector(2.33, 6.99, 0.0)
v4 = SVector(-2.36, 2.36, 0.0)
v5 = SVector(-6.99, 2.33, 0.0)
v6 = SVector(-2.33, 6.99, 0.0)
bc = SphericalBC(radius=8.0)
conf = Config{6}([v1,v2,v3,v4,v5,v6],bc)
d2mat = get_distance2_mat(conf)
vars = set_variables(conf,d2mat,runnerpot)
nnvtens = NNVT([4,2])
temp = TempGrid{2}(500,650)


state = MCState(temp.t_grid[1],temp.beta_grid[1],conf,nnvtens,runnerpot)
state2 = MCState(temp.t_grid[2],temp.beta_grid[2],conf,nnvtens,runnerpot)
refstate = MCState(temp.t_grid[1],temp.beta_grid[1],conf,nnvtens,runnerpot)

MCState{Float64, 6, SphericalBC{Float64}, NNPVariables2a{Float64, 6, 88}, NNVTVariables{Float64, 6, 4, 2}}(500.0, 631.5499705261944, Config{6, SphericalBC{Float64}, Float64}(SVector{3, Float64}[[2.36, 2.36, 0.0], [6.99, 2.33, 0.0], [2.33, 6.99, 0.0], [-2.36, 2.36, 0.0], [-6.99, 2.33, 0.0], [-2.33, 6.99, 0.0]], SphericalBC{Float64}(64.0)), [0.0 21.43780000000001 … 87.4234 43.433; 21.43780000000001 0.0 … 195.4404 108.578; … ; 87.4234 195.4404 … 0.0 43.431200000000004; 43.433 108.578 … 43.431200000000004 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 0.0, -0.2616528993354795, NNPVariables2a{Float64, 6, 88}([-0.07961990236909662, -0.046329508101558725, -0.06652413358790898, -0.08616269292006679, 0.012197910417455157, 0.0047854272256963905], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.09435530394181704 0.040032613593324516 … 0.0516772471800642 0.1181785536901605; 0.041829635204485544 0.0014182473523196017 … 0.02698557697421576 0.02698557697421576; … ; 0.012001828644531058 4.708405159025907e-7 … 0.00767972086

In [106]:
testgmat = MMatrix{88,6}(total_symm_calc(state.config.pos,d2mat,state.potential_variables.f_matrix,runnerpot.radsymfunctions,runnerpot.angsymfunctions,5,26,4,2)) 

88×6 MMatrix{88, 6, Float64, 528} with indices SOneTo(88)×SOneTo(6):
 0.0943553    0.0400326    0.0592781    0.0392187    0.0516772    0.118179
 0.0418296    0.00141825   0.0627722    0.123507     0.0269856    0.0269856
 0.151716     0.0648712    0.0840427    0.0629182    0.0626975    0.143403
 0.0315882    0.000341423  0.0772788    0.155697     0.0244546    0.0244546
 0.180692     0.0745949    0.0909785    0.0716025    0.0685224    0.15259
 0.0243412    0.000102788  0.0853434    0.17322      0.0206024    0.0206024
 0.204243     0.0747614    0.0792981    0.0678696    0.0734002    0.149061
 0.00672427   4.27814e-7   0.100464     0.207875     0.00659714   0.00659714
 0.0371199    0.0137437    0.0137449    0.00963622   0.0176255    0.0333012
 1.93379e-6   4.01273e-19  0.0195592    0.0439885    2.38658e-6   2.38658e-6
 ⋮                                                                ⋮
 0.0131774    0.000560929  0.0011261    0.000514664  1.90861e-9   0.00879061
 0.00187049   3.6269e-9    0.

In [107]:
Random.seed!(123)
generate_move!(state,"atommove")

MCState{Float64, 6, SphericalBC{Float64}, NNPVariables2a{Float64, 6, 88}, NNVTVariables{Float64, 6, 4, 2}}(500.0, 631.5499705261944, Config{6, SphericalBC{Float64}, Float64}(SVector{3, Float64}[[2.36, 2.36, 0.0], [6.99, 2.33, 0.0], [2.33, 6.99, 0.0], [-2.36, 2.36, 0.0], [-6.99, 2.33, 0.0], [-2.33, 6.99, 0.0]], SphericalBC{Float64}(64.0)), [0.0 21.43780000000001 … 87.4234 43.433; 21.43780000000001 0.0 … 195.4404 108.578; … ; 87.4234 195.4404 … 0.0 43.431200000000004; 43.433 108.578 … 43.431200000000004 0.0], [0.0, 21.420284581827794, 21.35915194132025, 22.300033538934237, 87.46519835414472, 43.37412319875922], 0.0, -0.2616528993354795, NNPVariables2a{Float64, 6, 88}([-0.07961990236909662, -0.046329508101558725, -0.06652413358790898, -0.08616269292006679, 0.012197910417455157, 0.0047854272256963905], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.09435530394181704 0.040032613593324516 … 0.0516772471800642 0.1181785536901605; 0.041829635204485544 0.0014182473523196017 … 0.02698557697421576 0.02698557

In [108]:
Random.seed!(123)
trialpos = atom_displacement(state.config.pos[1] , state.max_displ[1] , state.config.bc)
trialpos == state.ensemble_variables.trial_move

true

In [109]:
state = get_energy!(state,runnerpot,"atommove")
delen = state.new_en - state.en_tot
delen ≈ -7.5971516e-5
metropolis_condition("atommove",state,ensemble)
MCRun.swap_config!(state,"atommove")
state.ensemble_variables.trial_move == state.config.pos[1]

true

In [110]:
state.en_tot - state2.en_tot  ≈ delen
exc_trajectories!(state,state2)
state2.en_tot - state.en_tot  ≈ delen

true

In [111]:
state.config.pos == conf.pos

true

In [112]:
Random.seed!(123)

MCMoves.swap_atoms(state)

state.ensemble_variables.swap_indices

2-element SVector{2, Int64} with indices SOneTo(2):
 3
 6

In [113]:
state = get_energy!(state,runnerpot,"atomswap")

MCState{Float64, 6, SphericalBC{Float64}, NNPVariables2a{Float64, 6, 88}, NNVTVariables{Float64, 6, 4, 2}}(500.0, 631.5499705261944, Config{6, SphericalBC{Float64}, Float64}(SVector{3, Float64}[[2.36, 2.36, 0.0], [6.99, 2.33, 0.0], [2.33, 6.99, 0.0], [-2.36, 2.36, 0.0], [-6.99, 2.33, 0.0], [-2.33, 6.99, 0.0]], SphericalBC{Float64}(64.0)), [0.0 21.43780000000001 … 87.4234 43.433; 21.43780000000001 0.0 … 195.4404 108.578; … ; 87.4234 195.4404 … 0.0 43.431200000000004; 43.433 108.578 … 43.431200000000004 0.0], [0.0, 21.420284581827794, 21.35915194132025, 22.300033538934237, 87.46519835414472, 43.37412319875922], -0.2633613895303775, -0.2616528993354795, NNPVariables2a{Float64, 6, 88}([-0.07961990236909662, -0.046329508101558725, -0.06652413358790898, -0.08616269292006679, 0.012197910417455157, 0.0047854272256963905], [-0.08239162386142426, -0.04681777958397641, -0.06661685943374054, -0.0845674050065224, 0.01256101002919668, 0.0044712683260893955], [0.09435530394181704 0.040032613593324516

In [114]:
delen2 = state.new_en - state.en_tot
delen2 ≈ -0.0017084901948

true

In [115]:
MCRun.acc_test!(state,nnvtens,"atomswap")

88×6 MMatrix{88, 6, Float64, 528} with indices SOneTo(88)×SOneTo(6):
 0.0764098    0.0172475    0.036493     0.0571642    0.0766554    0.143157
 0.0684136    0.0351715    0.0965254    0.0969233    0.00108816   0.00108816
 0.116903     0.0442275    0.063399     0.0977312    0.0851454    0.165851
 0.0802448    0.0291942    0.106132     0.10704      0.000285994  0.000285994
 0.134339     0.0575585    0.0739422    0.117956     0.0866876    0.170755
 0.0876009    0.0233529    0.108593     0.10996      9.06819e-5   9.06819e-5
 0.138324     0.0702129    0.0747496    0.133788     0.0781374    0.153798
 0.103954     0.0067095    0.107173     0.110645     4.2065e-7    4.2065e-7
 0.0233787    0.0137425    0.0137437    0.0233775    0.017627     0.0333027
 0.0219943    1.93528e-6   0.0195612    0.0219962    4.94667e-19  4.94667e-19
 ⋮                                                                ⋮
 0.000660213  1.68307e-9   0.000564806  0.00610931   0.000636097  0.00942763
 0.0236009    0.00097379

In [116]:
state.dist2_mat[3,4] == refstate.dist2_mat[6,4]

true

In [117]:
refmat = copy(refstate.dist2_mat[6,:] )
refmat[6] = refstate.dist2_mat[6,3]
refmat[3] = 0.

0.0

In [119]:
refmat == state.dist2_mat[3,:]

true

In [101]:
state.en_tot - refstate.en_tot == delen2

true

In [104]:
state.config.pos[3] == refstate.config.pos[6]
state.config.pos[6] == refstate.config.pos[3]

true